In [1]:
import minsearch
import json

In [2]:
with open('documents.json') as f:
    docs_raw = json.load(f)

In [3]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [4]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [5]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"],
)

In [6]:
index.fit(documents)

In [7]:
q = "the course is already started, can I still join?"

In [11]:
boosts = {'question': 3.0, 'section': 0.5}
results = index.search(
    query=q,
    filter_dict={"course": "data-engineering-zoomcamp"},
    boost_dict=boosts,
    num_results=5
)

In [12]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineerin

In [13]:
from openai import OpenAI
client = OpenAI()

In [14]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": q},
    ]
)

In [15]:
response.choices[0].message.content

"It's possible that you can still join the course even if it has already started, but it depends on several factors such as the course policies, the instructor's discretion, and the institution's guidelines. Here are some steps you can take:\n\n1. **Check the Course Information:** Look at the course syllabus or description to see if there is any information about late enrollment.\n\n2. **Contact the Instructor:** Reach out to the course instructor via email or in person to inquire if it's too late to join. Explain your situation and express your interest in the course.\n\n3. **Speak with an Academic Advisor:** An academic advisor can provide information about the institution's policies regarding late enrollment and might help you with the process.\n\n4. **Evaluate the Workload:** Consider how much material you have missed and if you can realistically catch up without affecting the quality of your learning.\n\n5. **Understand the Deadlines:** Make sure you understand any deadlines for a

In [20]:
prompt_template = """
You're a course assistant helping a student with a question about the course. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the information provided in the CONTEXT.
If the CONTEXT does not contain answer, output NONE.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [17]:
context = ""

for doc in results:
    context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [19]:
print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1
Subscribe to course public Google Calendar (it works from Desktop only).
Register before the course starts using this link.
Join the course Telegram channel with announcements.
Don’t forget to register in DataTalks.Club's Slack and join the channel.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: 

In [21]:
prompt = prompt_template.format(question=q, context=context).strip()

In [22]:
print(prompt)

You're a course assistant helping a student with a question about the course. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the information provided in the CONTEXT.
If the CONTEXT does not contain answer, output NONE.

QUESTION: the course is already started, can I still join?

CONTEXT:
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1
Subscribe to course public Google Calendar (it w

In [23]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "user", "content": prompt},
    ]
)

response.choices[0].message.content

'Yes, you can still join the course even after it has started. Be aware that there will be deadlines for turning in the final projects, so try not to leave everything for the last minute.'